# SQL概念
* 查询语句结果集产生顺序，先是FROM，然后是WHERE，然后是GROUP BY, 然后是HAVING, 最后才是SELECT
    * FORM: 对FROM的左边的表和右边的表计算笛卡尔积。产生虚表VT1
    * ON: 对虚表VT1进行ON筛选，只有那些符合<join-condition>的行才会被记录在虚表VT2中。
    * JOIN： 如果指定了OUTER JOIN（比如left join、 right  join），那么保留表中未匹配的行就会作为外部行添加到虚拟表VT2中，产生虚拟表VT3, rug  from子句中包含两个以上的表的话，那么就会对上一个join连接产生的结果VT3和下一个表重复执行步骤1~3这三个步骤，一直到处理完所有的表为 止。
    * WHERE： 对虚拟表VT3进行WHERE条件过滤。只有符合<where-condition>的记录才会被插入到虚拟表VT4中。
    * GROUP BY: 根据group by子句中的列，对VT4中的记录进行分组操作，产生VT5.
    * CUBE | ROLLUP: 对表VT5进行cube或者rollup操作，产生表VT6.
    * HAVING： 对虚拟表VT6应用having过滤，只有符合<having-condition>的记录才会被 插入到虚拟表VT7中。
    * SELECT： 执行select操作，选择指定的列，插入到虚拟表VT8中。
    * DISTINCT： 对VT8中的记录进行去重。产生虚拟表VT9.
    * ORDER BY: 将虚拟表VT9中的记录按照<order_by_list>进行排序操作，产生虚拟表VT10.
    * LIMIT：取出指定行的记录，产生虚拟表VT11, 并将结果返回。
* 连接查询：
    * 组合查询如果不限制WHERE条件谓词，结果集就是两个表的笛卡尔乘积
    * 自然连接（内连接）：只选择两个表中属性相同并且值相等的集合，也就是组合查询中指定WHERE条件谓词，如：```WHERE A.id = B.id```或者```FROM A NATURAL JOIN B```或者指定谓词列```FROM A [INNER] JOIN B ON A.id=B.id```指定连接列```FROM A NATURAL JOIN B USING (column_name)```
    * 外连接，避免自然连接中数据丢失（自然连接中如果出现不匹配的数据则不会显示，但是外连接会保留），
        * 左外连接：保留左外连接左边关系中的元组```FROM A NATURAL LEFT OUTER JOIN B```，保留A中的元组
        * 右外连接：保留右外连接右边关系中的元组```FROM A NATURAL RIGHT OUTER JOIN B```，保留B中的元组
        * 全外连接：保留出现在两个关系中的元组```FROM A NATURAL FULL OUTER JOIN B```，保留A和B中的元组
* %：匹配任意子串，_：匹配任意一个字符，模式匹配的字符串是大小写敏感的
* 集合运算：UNION并集, INTERSECT交集, EXCEPT差集；所有集合操作都自动去重，如果不想去重就需要在集合操作后面加ALL关键字
* 分组查询的限制GROUP BY：出现在SELECT中的列，但是没出现在聚集函数中的列，必须全部在GROUP BY中出现；任何出现在HAVING子句中的列，但是没有出现在聚集函数中的列，必须全部在GROUP BY中出现
* HAVING字句：GROUP BY中的WHERE，对分组数据进行过滤筛选，谓词可以是分组结果中的单独的列，也可以是聚集函数的计算结果
    ```sql
    SELECT product_id, p.name, (sum(s.units) * (p.price - p.cost)) AS profit
    FROM products p LEFT JOIN sales s USING (product_id)
    WHERE s.date > CURRENT_DATE - INTERVAL '4 weeks'
    GROUP BY product_id, p.name, p.price, p.cost
    HAVING sum(p.price * s.units) > 5000;
    ```
* 子查询不仅可以使用IN，还可以是用(=, <, >, ...)SOME，(=, <, >, ...)ALL, EXISTS, UNIQUE，而且可以出现在WHERE, FROM, SELECT(必须只返回单列数据)中
* 完整约束性：
    * 单个关系的约束：NOT NULL, UNIQUE（A_1, A_2, A_3）, CHECK
    * 参照完整性：FOREIGN KEY (COL_NAME) REFERENCES TABLE_NAME
* 存储过程和函数实例：
    * 创建和调用：
    <table>
        <tr>
            <td><img src='../images/sql/创建函数.png'></td>
            <td><img src='../images/sql/调用函数.png'></td>
            <td><img src='../images/sql/创建存储过程.png'></td>
            <td><img src='../images/sql/调用存储过程.png'></td>
        </tr>
    </table>
    * 条件控制语句：
    <table>
        <tr>
            <td><img src='../images/sql/while循环.png'></td>
            <td><img src='../images/sql/for循环.png'></td>
            <td><img src='../images/sql/if语句.png'></td>
            <td><img src='../images/sql/声明异常.png'></td>
        </tr>
    </table>
    * 完整示列：<img src='../images/sql/函数示例代码.png' width='300px'>
* 触发器示列：<img src='../images/sql/触发器示列代码.png' width='350px'>
* 数据库三范式：
    * 基础概念：
        * 函数依赖：给定属性X，必能确定属性Y的值，则称Y函数依赖于X，X $\rightarrow$ Y
        * 完全函数依赖：如果X $\rightarrow$ Y，但是X的任何一个真子集不是Y的函数依赖，称Y对X完全函数依赖
        * 部分函数依赖：如果Y函数依赖X，但是Y不完全依赖X，则Y部分函数依赖X
        * 传递函数依赖：Z函数依赖于Y，且Y函数依赖于X（Y不包含于X，且X不函数依赖于Y），那么我们就称Z传递函数依赖于X
        * 码：设K为某表中的一个属性或属性组，若除K之外的所有属性都完全函数依赖于K，称K为候选码，简称为码，假如当K确定的情况下，该表除K之外的所有属性的值也就随之确定，那么K就是码，一张表中可以有超过一个码，主码就是主键
        * 非主属性：包含在任何一个码中的属性成为主属性，除主键以外的属性
    * 第一范式(1FN)：关系中的属性都不可再分，属性是原子性的，RDBMS中存在的表都符合1FN
        * 通俗的说：字段是最小的单元不可再分，即列不能够再分成其他几列；不能是复合属性、不能是多值属性、不能是重复属性；
        * 存在大量的数据冗余
        * 增、删、改存在异常
    * 第二范式(2FN)：在1FN的基础上，消除非主属性对于码的部分函数依赖
        * 通俗的说：表中的字段必须完全依赖于全部主键而非部分主键，即表必须有一个主键，没有包含在主键中的列必须完全依赖于主键，而不能只依赖于主键的一部分；表中必须存在侯选关键字，即每一行不同于其他任一行，是惟一区分的，任何非关键字段不能依赖于侯选关键字的一部分
        * 消除数据冗余，解决修改异常
        * 增、删存在异常
    * 第三范式(3FN)：在2FN的基础上，消除非主属性对于码的传递函数依赖
        * 通俗的说：非主键外的所有字段必须互不依赖，非关键字字段必须直接依赖任一侯选关键字，也就是说表中不能含有其他表的非主属性
        * 消除大量数据冗余，解决修改异常
        * 解决增、删异常
    * BC范式：在3NF的基础上，消除主属性对于码的部分与传递函数依赖，没有任何属性完全函数依赖于非候选码的任何一组属性
        * 通俗的说：每个表中只有一个候选键
    * 第四范式(4FN)：在BCNF的基础上，消除表中的多值依赖

****

# SQL 基础操作	
* 查看SQL执行计划，解释查询语句，调试SQL：```EXPLAIN SQL```或者```EXPLAIN PARTITIONS SQL```查询分区搜索
* 创建分区表语法：
    ```sql
    CREATE TABLE IF NOT EXISTS user_info ( 
        uuid BIGINT NOT NULL PRIMARY KEY
    ) ENGINE= InnoDB 
      PARTITION BY RANGE (uuid) (
          PARTITION p0 VALUES LESS THAN (10),
          PARTITION p1 VALUES LESS THAN (20));
    ```
* SET和SELECT：设置和查询全局变量
* 强行使用某个索引查询：```SELECT * FROM table FORCE INDEX(index_name)```
* 获取表的下一个自增长id值：```SELECT MAX(auto_inc_col_name) FROM table_name FOR UPDATE;```
* BENCHMARK()：测试语句执行速度
* 查询建表的语法解析：```SHOW CREATE TABLE;```
* 数据类型：括号的数字是单位是位
    * 整型：TINYINT(8), SMALLINT(16), MEDIUMINT(24), INT(32), BIGINT(64)
    * 实数：FLOAT(32), DOUBLE(64), DECIMAL(18, 9)占用72位，小数点两边各9个数字分别占用4字节32位，小数点占用一字节8位
    * 字符串：CHAR, VARCHAR
    * 字符串块：TINYTEXT, SMALLTEXT = TEXT, MEDIUMTEXT, LONGTEXT
    * 二进制字符串：BINARY, VARBINARY
    * 二进制字符串块：TINYBLOB, SMALLBLOB = BLOB, MEDIUMBLOB, LONGBLOB
    * 枚举：固定的字符串使用枚举，做CHECK约束
    * 日期：DATETIME(9999), TIMESTAMP(2038), FROM_UNIXTIME(), UNIX_TIMESTAMP()
    * 位数据：BIT, SET, FIND_IN_SET(), FIELD()
    * IP地址：INET_ATON(), INET_NTOA()转换成无符号整数存储
* 创建表：
    ```sql
    CREATE [TEMPORARY] TABLE [IF NOT EXISTS] tbl_name
        (create_definition,...)
        [table_options]
        [partition_options]
    CREATE TABLE new_tbl LIKE orig_tbl;
    ```
* 创建视图：
    ```sql
    CREATE VIEW view_name [(column_list)]
        AS select_statement
        [WITH [CASCADED | LOCAL] CHECK OPTION]
    ```
* 创建索引：
    ```sql
    CREATE [UNIQUE|FULLTEXT|SPATIAL] INDEX index_name
        [index_type]
        ON tbl_name (index_col_name,...)
        [index_option]
        [algorithm_option | lock_option]
    ```
* 授权操作：
    ```sql
    GRANT priv_type [(column_list)] [, priv_type [(column_list)]] ...
        ON [object_type] priv_level
        TO user [auth_option] [, user [auth_option]] ...
        [REQUIRE {NONE | tls_option [[AND] tls_option] ...}]
        [WITH {GRANT OPTION | resource_option} ...]
    GRANT ALL ON db1.* TO 'jeffrey'@'localhost';
    GRANT SELECT ON db2.invoice TO 'jeffrey'@'localhost';
    ```
* 取消授权：
    ```sql 
    REVOKE priv_type [(column_list)] [, priv_type [(column_list)]] ...
        ON [object_type] priv_level
        FROM user [, user]
    REVOKE INSERT ON *.* FROM 'jeffrey'@'localhost';
    ```
* 创建用户：
    ```sql
    CREATE USER [IF NOT EXISTS]
        user [auth_option] [, user [auth_option]] ...
        [REQUIRE {NONE | tls_option [[AND] tls_option] ...}]
        [WITH resource_option [resource_option] ...]
        [password_option | lock_option]
    CREATE USER 'jeffrey'@'localhost'
        IDENTIFIED WITH sha256_password BY 'new_password'
        PASSWORD EXPIRE INTERVAL 180 DAY;
    ```
* 创建函数和存储过程：
    ```sql
    CREATE [DEFINER = { user | CURRENT_USER }]
        PROCEDURE sp_name ([proc_parameter[,...]])
        [characteristic ...] routine_body

    CREATE [DEFINER = { user | CURRENT_USER }]
        FUNCTION sp_name ([func_parameter[,...]])
        RETURNS type
        [characteristic ...] routine_body
    ```
* 创建触发器：
    ```sql
    CREATE [DEFINER = { user | CURRENT_USER }]
        TRIGGER trigger_name
        trigger_time trigger_event
        ON tbl_name FOR EACH ROW
        [trigger_order]
        trigger_body
    ```

****

# 查询系统变量
* 用户访问权限查询：```SELECT * FROM mysql.user;```
* 查看数据库文件路径：```SHOW VARIABLES LIKE 'datadir';```
* 查询io吞吐量： ```SHOW VARIABLES LIKE 'innodb_io_capacity';```
* 日志查询：
    * 查询错误日志路径：```SHOW VARIABLES like 'log_error';```
    * 查询分析慢查询日志： ```SHOW VARIABLES like 'long_query_time'; SHOW VARIABLES like 'log_slow_%'; SHOW VARIABLES like 'log_queries_not_using_indexes';```；mysqldumpslow处理慢查询日志，解析工具pt-query-digest
    * 二进制日志，就是binlog，记录对数据库更改的所有操作，用于恢复和复制：```SHOW VARIABLES like 'log_bin'; SHOW VARIABLES like 'sync_binlog';```；mysqlbinlog处理二进制日志；跟重做日志的区别在于抽象层次，二进制日志属于mysql层次，重做日志属于innodb存储引擎，二进制日志记录的是具体的操作，重做日志记录的物理页的改变，二进制日志只有事物提交时才记录，重做日志一直在记录；
    * 查询重做日志状态：```SHOW VARIABLES like 'innodb_log_file_size'; SHOW VARIABLES like 'innodb_log_files_in_group'; SHOW VARIABLES like 'innodb_log_group_home_dir';```
* 存储引擎查询：
    * 查询某个数据库使用的存储引擎：```SELECT TABLE_NAME, ENGINE FROM information_schema.TABLES WHERE TABLE_SCHEMA = 'table_name';```
    * 查询默认存储引擎：```SHOW GLOBAL VARIABLES LIKE '%engine%';```
    * 查询mysql支持哪些存储引擎：```SHOW ENGINES;```
    * 查询存储引擎状态：```SHOW ENGINE innodb STATUS;```，结果中的BUFFER POOL AND MEMORY解释缓存池的使用状态
    * 存现存储引擎IO线程：```SHOW VARIABLES LIKE 'innodb_io_threads';```
    * 查询innodb版本信息：```SHOW VARIABLES LIKE 'innodb_version';```
* 查询缓存：
    * 查询缓存池大小：```SHOW VARIABLES LIKE 'innodb_buffer_pool_size';```
    * 查询重做日志缓存池大小：```SHOW VARIABLES LIKE 'innodb_log_buffer_size';```；
* 查询最大脏数据比例：```SHOW VARIABLES LIKE 'innodb_max_dirty_pages_pct';```
* 查询表空间文件路径和是否每个表空间独立使用一个文件：```SHOW VARIABLES like 'innodb_data_file_path'; SHOW VARIABLES like 'innodb_file_per_table';```

* 查询表约束：```SELECT * FROM information_schema.REFERENTIAL_CONSTRAINTS WHERE CONSTRAINT_SCHEMA = 'table_name';```
* 查询当前数据库的sql模式：```SHOW VARIABLES like 'sql_mode';```
* 视图查询：
    * 查询数据库中所有的视图：```SELECT * FROM information_schema.TABLES WHERE TABLE_TYPE = 'BASE TABLE' AND TABLE_SCHEMA = 'database_name';```
    * 查询数据库视图详情：```SELECT * FROM information_schema.VIEWS WHERE TABLE_SCHEMA = 'database_name';```
* 查询数据库表的分区情况：```SELECT * FROM information_schema.PARTITIONS WHERE  TABLE_SCHEMA = 'database_name';```
* 查询经过默认排序的列：```SHOW VARIABLES like 'sort_rows';```
* 查询从服务器状态：```SHOW SLAVE STATUS;```
* 查询主服务器状态：```SHOW MASTER STATUS;```
* 查询表的状态：```SHOW TABLE STATUS LIKE 'table_name';```
* 查询剖析工具：```SET PROFILING=1; SHOW PROFILES FOR QUERY [N]```
* 查看全局服务器配置信息：```SHOW GLOBAL STATUS;```
* 查看会话级别服务器配置信息：```SHOW STATUS;```